In [ ]:
import sys
!{sys.executable} -m pip install python-docx
!{sys.executable} -m pip install --upgrade transformers[torch] datasets evaluate sacremoses
!{sys.executable} -m pip install --upgrade mlxtend wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00
  Attempting uninst

In [ ]:
import docx
import os, re
from collections import Counter

# Data loading code

Counter(labels)


Counter({0: 168, 1: 195})

In [ ]:

from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(labels, labels)
dummy_clf.score(labels, labels)

0.5371900826446281

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import wandb, gc, os, pickle, datasets, transformers, torch, evaluate

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from datasets import concatenate_datasets, Dataset
from transformers import (
    BertConfig,
    AutoTokenizer,
    BertModel,
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments)

transformers.logging.set_verbosity_error()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def preprocess_function(examples, tokenizer):
    return tokenizer.batch_encode_plus(examples["text"], is_split_into_words=False, truncation=True, max_length=512, padding='max_length', pad_to_max_length=True)

model_name = "allegro/herbert-large-cased"
runtime_name = "herbert-large-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding="max_length", max_length=512, truncation=True, do_lower_case=False)


dataset_dict = {'label': labels, 'text': texts}
dataset_hf = Dataset.from_dict(dataset_dict)
dataset_hf = dataset_hf.map(lambda p: preprocess_function(p, tokenizer), batched=True)

labels = np.array(labels)
texts = np.array(texts)

true_labels = [] # ground truth
predictions = [] # model predictions

batch_size = 4
fold_nr=0
all_results = []
device = "cuda"

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(texts):

  print(f'Fold_nr= {fold_nr}') ; fold_nr += 1

  model_config = BertConfig.from_pretrained(model_name, num_labels=2)
  model = BertForSequenceClassification.from_pretrained(model_name, config=model_config)

  training_args = TrainingArguments(
      do_eval=True,
      output_dir='./results-'+runtime_name,          # output directory
      overwrite_output_dir=True,
      num_train_epochs=10,              # total number of training epochs
      per_device_train_batch_size=batch_size,   # 8 @A100 batch size per device during training
      per_device_eval_batch_size=batch_size,    # 8 @A100 batch size for evaluation
      warmup_steps=120,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=80,
      evaluation_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
      save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
      # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
      learning_rate=1e-5,
      eval_steps = 25,
      save_total_limit = 1,
      load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      metric_for_best_model='accuracy',
      disable_tqdm=False,
      run_name = runtime_name
  )

  train_dataset = dataset_hf.select(train_index)
  test_dataset = dataset_hf.select(test_index)

  # let's have validation set from 5% of train:
  train_dataset = train_dataset.train_test_split(test_size=0.05, shuffle=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=512, return_tensors='pt')

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset['train'],  # train part of train
      eval_dataset=train_dataset['test'],  # this is validation
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  trainer.train()

  predictions_obj = trainer.predict(test_dataset)
  y_pred = np.argmax(predictions_obj.predictions, axis=1)
  test_dataset = test_dataset.add_column("y_pred", y_pred)

  predictions.append( y_pred )
  true_labels.append( labels[test_index] )


print( classification_report(np.concatenate(true_labels).ravel().tolist(), np.concatenate(predictions).ravel().tolist() ) )


tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

...

Flattening the indices:   0%|          | 0/36 [00:00<?, ? examples/s]

Fold_nr= 9


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.699570,0.470588,0.181818,0.250000,0.142857
50,No log,0.737436,0.411765,0.583333,0.411765,1.000000
75,No log,0.689328,0.529412,0.000000,0.000000,0.000000
100,0.677400,0.640742,0.647059,0.571429,0.571429,0.571429
125,0.677400,0.692626,0.529412,0.500000,0.444444,0.571429
150,0.677400,0.664166,0.588235,0.666667,0.500000,1.000000
175,0.719600,0.621880,0.647059,0.500000,0.600000,0.428571
200,0.719600,0.629887,0.705882,0.736842,0.583333,1.000000
225,0.719600,0.561853,0.823529,0.823529,0.700000,1.000000
250,0.606900,0.657428,0.705882,0.666667,0.625000,0.714286


Flattening the indices:   0%|          | 0/36 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       168
           1       0.90      0.86      0.88       195

    accuracy                           0.88       363
   macro avg       0.88      0.88      0.88       363
weighted avg       0.88      0.88      0.88       363

